In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('교통량_202410101006.csv')

In [8]:

# 해리 (nautical mile)을 km로 변환
nautical_mile_to_km = 1.852

# 두 좌표 간의 거리 계산 함수 (Haversine formula)
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # 지구의 반지름 (km)
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # 거리 (km)

# 주어진 좌표가 반지름 내에 있는지 확인하는 함수
def is_within_radius(lat1, lon1, lat2, lon2, radius_nm):
    radius_km = radius_nm * nautical_mile_to_km  # 반지름을 km로 변환
    distance = haversine(lon1, lat1, lon2, lat2)
    return distance <= radius_km

# 좌표가 주어진 경계(선) 내에 있는지 확인하는 함수
def is_within_area(lat, lon, start_lat, start_lon, end_lat, end_lon):
    return min(start_lat, end_lat) <= lat <= max(start_lat, end_lat) and min(start_lon, end_lon) <= lon <= max(start_lon, end_lon)

# 첫 번째 범위 (오륙도 및 생도 기준)
orukdo_lat, orukdo_lon = 35 + 5/60 + 28/3600, 129 + 7/60 + 36/3600
orukdo_end_lat, orukdo_end_lon = 35 + 20/60 + 21/3600, 129 + 24/60 + 1.5/3600
saengdo_lat, saengdo_lon = 35 + 2/60 + 13.7/3600, 129 + 5/60 + 34.8/3600
saengdo_end_lat, saengdo_end_lon = 34 + 45/60 + 30/3600, 128 + 52/60 + 5/3600

# 두 번째 범위 (가덕도 기준)
gadeokdo_lat, gadeokdo_lon = 34 + 59/60 + 22.2/3600, 128 + 49/60 + 44.6/3600
gadeokdo_end_lat1, gadeokdo_end_lon1 = 35 + 3/60 + 59/3600, 129 + 13/60 + 17/3600
gadeokdo_end_lat2, gadeokdo_end_lon2 = 34 + 43/60 + 9/3600, 128 + 36/60 + 15/3600

# 세 번째 범위 (팔미도 기준, 원형 범위)
palmi_lat, palmi_lon = 37 + 21/60 + 29.8/3600, 126 + 30/60 + 38.9/3600
radius_nm = 20  # 반지름 20해리

# 첫 번째 범위에 좌표가 포함되는지 확인하는 함수 (오륙도와 생도)
def check_in_first_area(lat, lon):
    return (is_within_area(lat, lon, orukdo_lat, orukdo_lon, orukdo_end_lat, orukdo_end_lon) or
            is_within_radius(lat, lon, orukdo_lat, orukdo_lon, radius_nm) or
            is_within_area(lat, lon, saengdo_lat, saengdo_lon, saengdo_end_lat, saengdo_end_lon) or
            is_within_radius(lat, lon, saengdo_lat, saengdo_lon, radius_nm))

# 두 번째 범위에 좌표가 포함되는지 확인하는 함수 (가덕도)
def check_in_second_area(lat, lon):
    return (is_within_area(lat, lon, gadeokdo_lat, gadeokdo_lon, gadeokdo_end_lat1, gadeokdo_end_lon1) or
            is_within_area(lat, lon, gadeokdo_lat, gadeokdo_lon, gadeokdo_end_lat2, gadeokdo_end_lon2) or
            is_within_radius(lat, lon, gadeokdo_lat, gadeokdo_lon, radius_nm))

# 세 번째 범위에 좌표가 포함되는지 확인하는 함수 (팔미도)
def check_in_third_area(lat, lon):
    return is_within_radius(lat, lon, palmi_lat, palmi_lon, radius_nm)

def determine_area(lat, lon):
    in_first_area = check_in_first_area(lat, lon)
    in_second_area = check_in_second_area(lat, lon)
    
    # 두 범위 모두 속하는 경우를 4로 구분
    if in_first_area and in_second_area:
        return 4  # 겹치는 경우
    elif in_first_area:
        return 1
    elif in_second_area:
        return 2
    elif check_in_third_area(lat, lon):
        return 3
    else:
        return 0

# 다시 데이터 처리
df['범위'] = df.apply(lambda row: determine_area(row['위도'], row['경도']), axis=1)

# 교통량이 60 이상인 데이터만 필터링
df_filtered = df[df['교통량(척)'] >= 60]

# 결과를 다시 저장
output_file_filtered = 'zzin_last_access.csv'
df_filtered.to_csv(output_file_filtered, index=False)

In [20]:
df1 = pd.read_csv('zzin_last_access.csv')
df1[df1['범위']==4]

,Unnamed: 0,격자번호,위경도,교통량(척),밀집도(%),어선,여객선,화물선,유조선,예인선,...,3t ~ 5t,5t ~ 10t,10t ~ 50t,50t ~ 100t,100t ~ 500t,500t이상,미상,위도,경도,범위
2,2,GR3_G3B33_L,"35.125, 129.075",122,83.25,2,3,16,47,13,...,0,0,1,3,25,18,75,35.125,129.075,4
3,3,GR3_G3B33_P,"35.075, 129.025",118,25.48,39,1,6,11,11,...,4,4,7,1,13,4,84,35.075,129.025,4
7,7,GR3_G3A44_T,"35.075, 128.975",96,22.94,60,1,8,1,7,...,10,2,1,0,5,1,39,35.075,128.975,4
26,26,GR3_G3A43_T,"35.075, 128.725",70,0.69,69,0,0,0,0,...,4,29,0,0,0,0,2,35.075,128.725,4
31,31,GR3_G3B33_Q,"35.075, 129.075",65,20.68,24,0,12,3,2,...,5,4,2,1,1,4,35,35.075,129.075,4
